<a href="https://colab.research.google.com/github/varunpothu/Smart_HealthCare_Chatbot/blob/main/symptom_to_medicine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting the Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Installing all requied libraries

In [2]:
# Install necessary libraries
!pip install transformers torch pandas streamlit pyngrok scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


Ngrok Autoken

In [3]:
# Authenticate Ngrok with your token
!ngrok config add-authtoken 2pPynQ1umOErGUuiyW31IDnmrF1_6VvhjMYosxEu6cnK5g3jx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# Importing Libraries

In [4]:
# Imports
import pandas as pd
import numpy as np
import json
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import pickle
from pyngrok import ngrok
import os

### Loading Datasets

In [5]:
# Load datasets
file1_path = '/content/drive/MyDrive/1.csv'
file5_path = '/content/drive/MyDrive/5.csv'
data1 = pd.read_csv(file1_path)
data5 = pd.read_csv(file5_path)

<ipython-input-5-3bf429b7f95e>:5: DtypeWarning: Columns (42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data5 = pd.read_csv(file5_path)



### Preprocessing the data

In [6]:
# Combine substitute columns
substitute_cols = [col for col in data5.columns if 'substitute' in col.lower()]
if substitute_cols:
    data5['substitutes'] = data5[substitute_cols].apply(lambda row: ', '.join(row.dropna()), axis=1)
    data5 = data5.drop(columns=substitute_cols)
else:
    data5['substitutes'] = 'Not available'



In [7]:
# Combine sideEffect columns
side_effect_cols = [col for col in data5.columns if 'sideEffect' in col.lower()]
if side_effect_cols:
    data5['side_effects'] = data5[side_effect_cols].apply(lambda row: ', '.join(row.dropna()), axis=1)
    data5 = data5.drop(columns=side_effect_cols)
else:
    data5['side_effects'] = 'Not available'


In [8]:
# Combine use columns
use_cols = [col for col in data5.columns if 'use' in col.lower()]
if use_cols:
    data5['uses'] = data5[use_cols].apply(lambda row: ', '.join(row.dropna()), axis=1)
    data5 = data5.drop(columns=use_cols)
else:
    data5['uses'] = 'Not available'

In [9]:
# Drop unnecessary columns
data5 = data5.drop(columns=['id', 'Habit Forming'], errors='ignore')


### Creating Mappings

In [10]:
# Create mappings
use_name_mapping = data5[['uses', 'name']].drop_duplicates().set_index('uses')['name'].to_dict()

# Save mappings for later use
with open('use_name_mapping.json', 'w') as f:
    json.dump(use_name_mapping, f, indent=4)


Bert Model

In [11]:
# BERT-based model
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

def get_bert_embedding(text):
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()

symptom_embeddings = {symptom: get_bert_embedding(symptom) for symptom in use_name_mapping.keys()}

with open('symptom_embeddings.pkl', 'wb') as f:
    pickle.dump(symptom_embeddings, f)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

# TF-IDF-based recommender

In [12]:
# TF-IDF-based recommender
class MedicineRecommender:
    def __init__(self, data):
        self.data = data
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(data['uses'])

    def recommend(self, user_input, top_n=3):
        user_vector = self.vectorizer.transform([user_input])
        similarity_scores = cosine_similarity(user_vector, self.tfidf_matrix)
        top_indices = similarity_scores.argsort()[0][-top_n:][::-1]
        recommendations = self.data.iloc[top_indices][['name', 'uses']]
        return recommendations

recommender = MedicineRecommender(data5)

with open('medicine_recommender.pkl', 'wb') as f:
    pickle.dump(recommender, f)


### Deployment Using Streamlit

In [13]:

# Saving Streamlit app
with open("app.py", "w") as f:
    f.write('''
import streamlit as st
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define the MedicineRecommender class
class MedicineRecommender:
    def __init__(self, data):
        self.data = data
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(data['uses'])

    def recommend(self, user_input, top_n=3):
        user_vector = self.vectorizer.transform([user_input])
        similarity_scores = cosine_similarity(user_vector, self.tfidf_matrix)
        top_indices = similarity_scores.argsort()[0][-top_n:][::-1]
        recommendations = self.data.iloc[top_indices][['name', 'uses']]
        return recommendations

# Load models and data
with open('symptom_embeddings.pkl', 'rb') as f:
    symptom_embeddings = pickle.load(f)

with open('medicine_recommender.pkl', 'rb') as f:
    recommender = pickle.load(f)

data5 = pd.read_csv('/content/drive/MyDrive/5.csv')

# Emergency detection
def is_emergency(symptoms):
    emergency_symptoms = [
        "chest pain", "severe bleeding", "difficulty breathing", "sudden confusion",
        "weakness or numbness on one side", "loss of consciousness", "severe headache",
        "seizures", "severe burns", "uncontrolled vomiting", "high fever", "persistent dizziness",
        "major trauma", "heart attack", "stroke", "difficulty speaking", "severe allergic reaction",
        "intense abdominal pain", "continuous chest pressure", "sudden vision loss",
        "high or low blood sugar", "severe dehydration", "painful swelling", "sudden severe back pain",
        "persistent vomiting with blood", "sepsis symptoms", "head trauma", "difficulty walking"
    ]
    for emergency in emergency_symptoms:
        if emergency in symptoms.lower():
            return True
    return False

def chatbot_response(symptoms):
    if is_emergency(symptoms):
        return "This is an emergency. Please consult a healthcare professional immediately."

    recommendations = recommender.recommend(symptoms, top_n=1)
    if recommendations.empty:
        return "No medicines found for the provided symptoms. Please consult a doctor."

    medicine = recommendations.iloc[0]
    side_effects = data5[data5['name'] == medicine['name']]['side_effects'].iloc[0] if 'side_effects' in data5.columns else 'Not available'
    substitutes = data5[data5['name'] == medicine['name']]['substitutes'].iloc[0] if 'substitutes' in data5.columns else 'Not available'
    therapeutic_class = data5[data5['name'] == medicine['name']]['Therapeutic Class'].iloc[0] if 'Therapeutic Class' in data5.columns else 'Not available'
    manufacturer = data5[data5['name'] == medicine['name']]['Manufacturer'].iloc[0] if 'Manufacturer' in data5.columns else 'Not available'

    return {
        "Medicine Name": medicine['name'],
        "Uses": medicine['uses'],
        "Side Effects": side_effects,
        "Substitutes": substitutes,
        "Therapeutic Class": therapeutic_class,
        "Manufacturer": manufacturer
    }

# Streamlit interface
st.title("Healthcare Medicine Recommendation Chatbot")
user_input = st.text_input("Enter your symptoms:")

if user_input:
    response = chatbot_response(user_input)
    if isinstance(response, dict):
        st.write("### Medicine Details:")
        for key, value in response.items():
            st.write(f"**{key}:** {value}")
    else:
        st.write(response)
    ''')


Using Ngrok for testing

In [15]:

# Start Streamlit and expose it using Ngrok
from threading import Thread

def run_streamlit():
    os.system("streamlit run app.py --server.port 8501")

thread = Thread(target=run_streamlit)
thread.start()

public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")


Streamlit app is running at: NgrokTunnel: "https://a1e4-34-125-11-192.ngrok-free.app" -> "http://localhost:8501"
